### Query showing Jobs, tasks, DBU and Cost

In [0]:
%sql
SELECT
  usage_metadata.job_id,
  run_id,
  jrl.period_start_time as jrl_period_start_time,
  u.sku_name,
  billing_origin_product,
  usage_date,
  usage_quantity AS `DBUs`,
  usage_quantity * pricing.default as usd_cost,
  elapsed_run_minutes,
  job_parameters,
  result_state,
  run_name
FROM
  system.billing.usage u
  
  join (select job_id,
              run_id, 
              timestampdiff(MINUTE, period_start_time, period_end_time) as elapsed_run_minutes, 
              result_state, 
              period_start_time ,
              job_parameters,
              job_parameters['PROFILE']['value'] as run_name
        from system.lakeflow.job_run_timeline
        ) 
      jrl on jrl.run_id  = u.usage_metadata.job_run_id 
      and jrl.job_id = u.usage_metadata.job_id
  
  LEFT JOIN system.billing.list_prices pricing ON u.sku_name = pricing.sku_name
      AND pricing.price_start_time <= u.usage_start_time
      AND (pricing.price_end_time >= u.usage_start_time OR pricing.price_end_time IS NULL)
      AND pricing.currency_code="USD"
WHERE
  result_state = 'SUCCEEDED' 
  --run_name in ('batch_aggr', 'batch_ingest')
  --and result_state = 'SUCCEEDED' 
GROUP BY ALL;

In [ ]:
with jobs_usage AS (
  SELECT
    *,
    usage_metadata.job_id,
    usage_metadata.job_run_id as run_id,
    identity_metadata.run_as as run_as
  FROM system.billing.usage
  WHERE billing_origin_product="JOBS"
),
jobs_usage_with_usd AS (
  SELECT
    jobs_usage.*,
    usage_quantity * pricing.default as usage_usd
  FROM jobs_usage
    LEFT JOIN system.billing.list_prices pricing ON
      jobs_usage.sku_name = pricing.sku_name
      AND pricing.price_start_time <= jobs_usage.usage_start_time
      AND (pricing.price_end_time >= jobs_usage.usage_start_time OR pricing.price_end_time IS NULL)
      AND pricing.currency_code="USD"
)